In [37]:
!pip3 install newsapi-python

You should consider upgrading via the '/Users/pavanibr/.pyenv/versions/3.9.6/bin/python3.9 -m pip install --upgrade pip' command.


In [38]:
# !python3 -m spacy download en_core_web_lg

In [39]:
from tqdm import tqdm
import pandas as pd
import numpy as np
from newsapi import NewsApiClient
import nltk
from nltk.tokenize import word_tokenize
import string
from nltk.corpus import stopwords
import spacy
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
import seaborn as sns
nlp = spacy.load('en_core_web_sm')

In [40]:
df = pd.read_csv('news dataset.csv', usecols=range(1, 3))

In [41]:
df.drop_duplicates('description', inplace=True)

In [42]:
descriptions = df['description'].values

In [43]:
def pre_process_text(text):

    tokens = word_tokenize(text)
    # convert to lower case
    tokens = [w.lower() for w in tokens]
    # remove punctuation from each word

    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    words = [word for word in stripped if word.isalpha()]
    # filter out stop words

    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    return words
  

def ner_tagging(text):
    tokens = word_tokenize(text)
    # convert to lower case
    tokens = [w.lower() for w in tokens]
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]

    words = [word for word in stripped if word.isalpha()]

    doc = nlp(text)
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    return words


In [44]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
processed_descriptions = []
triggered_events = []
for description in tqdm(descriptions):
    processed_descriptions.append(' '.join(pre_process_text(description)))

# Loop through the news articles and extract events
# for article in news_articles:
    # Parse the text with spaCy
    doc = nlp(description)
    
    # Extract events using spaCy's entity and dependency parsing
    for sent in doc.sents:
        for ent in sent.ents:
            if ent.label_ == 'ORG' or ent.label_ == 'PERSON':
                # Identify the subject of the sentence
                subj = ent.text
                # print(subj)
                verb = None
                obj = None
                for token in sent:
                    if token.dep_ == 'ROOT':
                        verb = token.text
                    if token.dep_ == 'dobj':
                        obj = token.text
                # Print the extracted event
                if verb and obj:
                    if sent not in triggered_events:
                      triggered_events.append(sent)

[nltk_data] Downloading package punkt to /Users/pavanibr/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/pavanibr/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
100%|██████████| 2128/2128 [03:45<00:00,  9.45it/s]


In [45]:
print(len(triggered_events)," and ",len(processed_descriptions))
print(type(processed_descriptions[0]))




2913  and  2128
<class 'str'>


In [46]:
nlp = spacy.load('en_core_web_sm')

sent_vecs = {}
docs = []

for index, description in enumerate(tqdm(processed_descriptions)):
    doc = nlp(description)
    docs.append(doc)
    sent_vecs[index] = doc.vector

100%|██████████| 2128/2128 [02:05<00:00, 16.93it/s]


In [47]:
vectors = list(sent_vecs.values())

In [48]:
vectors = np.array(vectors)

In [49]:
labels_results = {}
for i in tqdm(np.arange(0.001, 1, 0.002)):
    dbscan = DBSCAN(eps=i, min_samples=5, metric='cosine').fit(vectors)
    labels_results[i] = len(pd.Series(dbscan.labels_).value_counts())

100%|██████████| 500/500 [00:40<00:00, 12.34it/s]


In [50]:
dbscan = DBSCAN(eps=0.015, min_samples=5, metric='cosine').fit(vectors)

In [51]:
results = pd.DataFrame({
    'desc': processed_descriptions, 
    'label': dbscan.labels_
})

In [52]:
results['label'].value_counts()

label
-1    1711
 0     400
 1      12
 2       5
Name: count, dtype: int64

In [53]:
for index in results[results['label'] == 0].index[:6]:
    print(results.loc[index]['desc'],end='\n\n')
    # print('....')

chelsea sack mutu chelsea sacked adrian mutu failed drugs test yearold tested positive banned substance later denied cocaine october chelsea decided write possible transfer fee mutu signing parma last season may face twoyear suspension statement chelsea explaining decision readwe want make clear chelsea zero tolerance policy towards drugs mutu scored six goals first five games arriving stamford bridge form went decline frozen coach jose mourinho chelseas statement added applies performanceenhancing drugs socalled recreational drugs place club sport coming decision case chelsea believed clubs social responsibility fans players employees stakeholders football regarding drugs important major financial considerations company player takes drugs breaches contract club well football association rules club totally supports fa strong action drugs cases fifas disciplinary code stipulates first doping offence followed sixmonth ban sports world governing body reiterated stance mutus failed drugs t

In [54]:
results.head()

,desc,label
0,chelsea sack mutu chelsea sacked adrian mutu f...,0
1,record fails lift lacklustre meet yelena isinb...,-1
2,edu describes tunnel fracas arsenals edu lifte...,-1
3,ogara revels ireland victory ireland flyhalf r...,-1
4,unclear future striker baros liverpool forward...,-1


In [55]:
import spacy

nlp = spacy.load('en_core_web_sm')

news_articles = [
    'Moldova’s unrecognised breakaway region of Transnistria says it wants Moscow to bulk up its small contingent of peacekeepers because of what it called growing security risks, Russia’s state-owned RIA news agency reported. Though Moldova does not allow Russia to deploy new troops in Transnistria, Russia has had hundreds of peacekeepers in the region since a bloody war between pro-Russian separatists and Moldovan government forces after the 1991 Soviet breakup. “As long as Russia’s peacekeeping mission continues, Moldova is constrained in any military plans and preparations against Transnistria,” Leonid Manakov, the region’s envoy to Moscow, was cited as saying by RIA. “Transnistria has repeatedly applied for an increase in the number of Russian peacekeepers … There is such an option, and it is justified in terms of the worsening security risks,” he was quoted as saying. Moldova’s ties with Russia are currently badly strained and have deteriorated rapidly over the course of Moscow’s full-scale invasion of neighbouring Ukraine, which Chisinau has repeatedly condemned.',
    # 'The majority judges held that parliament which is competent to destroy a State cannot be held, on the theory of absolute sovereignty of the States, to be incompetent to acquire by legislation the property owned by the States. Even if the constitution were held to be a Federation and the States regarded qua the Union as sovereign, the power of the Union to legislate in respect of the property situated in the States would remain unrestricted, they held. It shows that the majority judges too believed that it is not incorrect to suggest that States and the Union can be regarded as sovereign together. ',
    # 'What appears to militate against the theory regarding the sovereignty of the State is the wide power with which the Parliament is invested to alter the boundaries of States, and even to extinguish the existence of  a State. There is no Constitutional guarantee against alteration of the boundaries of the States. By Article 2 of the Constitution, the Parliament may admit into the Union or establish new States on such terms and conditions as it thinks fit, and by Article 3 the Parliament is by law authorised to form a new State by redistribution of the territory of a State or by uniting two or more States or parts of States or by uniting any territory to a part of any State, increase the area of any State, diminish the area of any State, alter the boundaries of any State, and alter the name of any State',
    # 'The majority judges held that there could be no doubt that if the Union did so, it would not be using but abusing its power of acquisition, but the fact that a power is capable of being abused has never been in law a reason for denying its existence, for its existence has to be determined on very different considerations.',
    # '',
    '',
]

events_ = []

# Loop through the news articles and extract events
for article in news_articles:
    # Parse the text with spaCy
    doc = nlp(article)
    
    # Extract events using spaCy's entity and dependency parsing
    for sent in doc.sents:
    
        for ent in sent.ents:
            if ent.label_ == 'ORG' or ent.label_ == 'PERSON':
                # Identify the subject of the sentence
                subj = ent.text
                print(subj)
                verb = None
                obj = None
                for token in sent:
                    if token.dep_ == 'ROOT':
                        verb = token.text
                    if token.dep_ == 'dobj':
                        obj = token.text
                # Print the extracted event
                if verb and obj:
                    if sent not in events_:
                      events_.append(sent)
                    # print(f'{subj} {verb} {obj}')


RIA news agency
Moldova
Moldovan
Transnistria
Leonid Manakov
RIA
Transnistria
Moldova
Chisinau


In [56]:
print(len(events_))
strng = []
for eventss in events_:
  # eventss = (eventss).astype(str)
  strng.append(''.join(eventss.text))


3


In [57]:
pred_vecs = {}
pred_docs = []
for index, description in enumerate(tqdm(strng)):
    print(index)
    doc = nlp(description)
    pred_docs.append(doc)
    pred_vecs[index] = doc.vector

vectors_c = list(pred_vecs.values())
vectors_c = np.array(vectors_c)
# classifier results = dbscan.fit_predict(vectors_c)


100%|██████████| 3/3 [00:00<00:00, 76.80it/s]

0
1
2


In [58]:
print(vectors_c.shape)

(3, 96)


In [59]:
def dbscan_predict(model, X):

    nr_samples = X.shape[0]
    # print(nr_samples)
    y_new = np.ones(shape=nr_samples, dtype=int) * -1

    for i in range(nr_samples):
        diff = model.components_ - X[i, :]  # NumPy broadcasting

        dist = np.linalg.norm(diff, axis=1)  # Euclidean distance

        shortest_dist_idx = np.argmin(dist)

        if dist[shortest_dist_idx] < model.eps:
            y_new[i] = model.labels_[model.core_sample_indices_[shortest_dist_idx]]

    return y_new

In [60]:
output = dbscan_predict(dbscan,vectors_c)

In [61]:
prediction_results = pd.DataFrame({
    'desc': strng, 
    'label': output
})

In [62]:
import pandas as pd
# Filter the DataFrame for cluster 1
cluster_1_descriptions = results[results['label'] == 1]['desc']
print("Cluster 1 Descriptions:")
for description in cluster_1_descriptions:
    print(description)
    print("---")  # Separator for readability

Cluster 1 Descriptions:
scotland v italy sat murrayfield edinburgh saturday february gmt bbc five live website victory azzurri rome last year saw scotland end campaign without victory pressure scotland coach matt williams seeks first six nations victory eighth attempt italy lost opening games home ireland wales travel edinburgh high hopes coach john kirwan warned side must eradicate errors blighted loss wales however risk suffering third successive defeat defeat wales taught us anything level cant make mistakes kirwan said six nations every error make come high price aggressive minutes keep calm every situation display great maturity pitch fundamental keep cool difficult moments key situations game kirwan recalled experienced cristian stoica centre drafted david dal maso openside star flanker mauro bergamasco ruled rest tournament scotland also made two changes simon webster replacing simon danielli wing simon taylor returning first test year injured flanker jason white taylors recover

In [63]:
import pandas as pd

# Filter the DataFrame for cluster 2
cluster_2_descriptions = results[results['label'] == 2]['desc']
print("\nCluster 2 Descriptions:")
for description in cluster_2_descriptions:
    print(description)
    print("---")  



Cluster 2 Descriptions:
straw backs ending china embargo uk foreign secretary jack straw defended plans end european unions arms embargo china despite opposition us japan mr straw visiting beijing noted arms embargoes applied china burma zimbabwe north korea said terrible rights record eu imposed arms ban china troops opened fire protestors tiananmen square mr straw also signed deal chinauk tourism expected would increase number chinese tourists per year providing revenue china past said sees weapons ban politically driven want lifted order buy weapons mr straw speaking joint news conference chinese foreign minister li zhaoxing stressed point result decision lift arms embargo increase arms exports european union member states china either quantitative qualitative terms mr straw said earlier week said expected embargo lifted within six months mr straw faces tough opposition move tory foreign affairs spokesman michael ancram said lifting arms embargo would irresponsible would damage bri

In [64]:
prediction_results.head()

,desc,label
0,Moldova’s unrecognised breakaway region of Tra...,-1
1,Though Moldova does not allow Russia to deploy...,-1
2,Moldova’s ties with Russia are currently badly...,-1
